In [1]:
import pandas as pd
import networkx as nx
import igraph as ig
# import leidenalg
import numpy as np
import os
import gc
import ast
import hashlib
from collections import defaultdict, Counter
from itertools import combinations
import json
import ijson
from datetime import datetime
import pickle

In [3]:
from project_cda.anime_graph_builder import AnimeGraphBuilder
from project_cda.community_tracker import CommunityTracker
from project_cda.cluster_evaluation import ClusterEvaluation
from project_cda.partition_enricher import PartitionEnricher
from project_cda.cluster_visualizer import ClusterVisualizer

In [4]:
os.chdir("/home/yaroslav/FCUL/MARS_1.0")

In [5]:
# Настройка отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)


DATA_DIR = "data"
# INPUT PATHES
USERS_PATH = "data/datasets/anime_azathoth42/users_sterilized.csv"
USER_DICT_PATH = "data/helpers/user_dict_filtered.json"
ANIME_META_PATH = "data/datasets/anime_azathoth42/anime_sterilized.csv"

# OUTPUT PATHES
REPORTS_DIR = "data/reports"
PARTITIONS_DIR = f"data/partitions"
PLOTS_DIR = f"data/plots"

# Создаем структуру папок
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(PARTITIONS_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

In [ ]:
print(">>> Initializing Metadata & Builders...")
# 1. Загружаем метаданные (Enricher сразу распарсит жанры)
enricher = PartitionEnricher(ANIME_META_PATH)
# 2. Подготовка словаря для Evaluation (чистая копия для метрик)
# Превращаем DataFrame в словарь {id: {'genres': {...}, 'source': '...'}}
anime_info_clean = enricher.meta_df.set_index('anime_id')[['genres', 'source']].to_dict('index')
builder = AnimeGraphBuilder(USERS_PATH, USER_DICT_PATH, ANIME_META_PATH)

1. Методы кластеризации (ALGO)

    ------------------------------------------------------------------------------
    **leiden_mod** — Leiden (Modularity). Классика. Ищет крупные сообщества.

    *partition_object = g_igraph.community_leiden(objective_function="modularity", weights=weights, n_iterations=-1)*

    ------------------------------------------------------------------------------
    **leiden_cpm** — Leiden (CPM). Ищет плотные ядра. Требует RESOLUTION (начни с 0.01–0.05).
    
    *partition_object = g_igraph.community_leiden(objective_function="cpm", weights=weights, resolution_parameter=resolution, n_iterations=-1)*

    ------------------------------------------------------------------------------
    **infomap** — Infomap. Поток информации. Лучше всего находит границы жанров.

    *partition_object = g_igraph.community_infomap(edge_weights=weights)*

    ------------------------------------------------------------------------------
    **label_prop** — Label Propagation. Быстро, грязно, нестабильно (Baseline).

    *partition_object = g_igraph.community_label_propagation(weights=weights)*

    ------------------------------------------------------------------------------
    **eigenvector** — Leading Eigenvector. Спектральный метод. Медленнее.

    *partition_object = g_igraph.community_leading_eigenvector(weights=weights)*

    ------------------------------------------------------------------------------
    **walktrap** — Walktrap. Случайные блуждания (стабильнее, чем LabelProp).

    *dendrogram = g_igraph.community_walktrap(weights=weights)*

    *partition_object = dendrogram.as_clustering()*

2. Методы прореживания (SPARS_TYPE)

    ------------------------------------------------------------------------------
    **knn** — Оставить K лучших соседей (напр. 20). Гарантирует связность.

    *with open(gpickle_path, "rb") as f: G = pickle.load(f)*

    *sparse_graph = sparsify_knn(self, gpickle_path, k: int = 20)*

    ------------------------------------------------------------------------------
    **backbone** — Оставить статистически значимые связи (ALPHA ≈ 0.05). Может развалить граф.


    *with open(gpickle_path, "rb") as f: G = pickle.load(f)*

    *sparse_graph = sparsify_backbone(G, weight="weight", alpha=0.05, weight_threshold=1, save_path=None)*

    ------------------------------------------------------------------------------
    Без чистки (только если порог веса высокий).

### 3. Методы формирования графа: ***build_edges(year, max_users=100000, method="jaccard")***

    method = **jaccard | raw**

### 4. Именование файла (EXP_TAG)

    Стандарт: {Algo}_{Graph}_{Spars}_{Track}

    Пример: Infomap_Jac0.05_KNN20_Tr0.1

In [ ]:
import igraph as ig

def get_partition(g: ig.Graph, method_code: str, weights=None):
    """
    Фабрика алгоритмов.
    Принимает коды: 'leiden_mod', 'leiden_cpm_0.05', 'infomap', 'label_prop', 'walktrap'.
    """
    if g.vcount() == 0: return None
    
    # 1. LEIDEN MODULARITY (Стандарт)
    if method_code == "leiden_mod":
        return g.community_leiden(
            objective_function="modularity", weights=weights, n_iterations=-1
        )

    # 2. LEIDEN CPM (С параметром resolution)
    # Код вида: 'leiden_cpm_0.05'
    elif method_code.startswith("leiden_cpm"):
        try:
            res_val = float(method_code.split("_")[-1])
        except:
            res_val = 0.01
        return g.community_leiden(
            objective_function="cpm", weights=weights, 
            resolution_parameter=res_val, n_iterations=-1
        )

    # 3. INFOMAP (Поток)
    elif method_code == "infomap":
        return g.community_infomap(edge_weights=weights)

    # 4. LABEL PROPAGATION (Быстро)
    elif method_code == "label_prop":
        return g.community_label_propagation(weights=weights)
        
    # 5. WALKTRAP (Случайные блуждания, альтернатива Infomap)
    elif method_code == "walktrap":
        # steps=4 - дефолт, можно не менять
        dendrogram = g.community_walktrap(weights=weights, steps=4)
        return dendrogram.as_clustering()
        
    # 6. EIGENVECTOR (Спектральный)
    elif method_code == "eigenvector":
        return g.community_leading_eigenvector(weights=weights)

    else:
        raise ValueError(f"Unknown method: {method_code}")

def generate_exp_key(algo, graph_type, edge_th, spars_type, spars_param, track_th):
    """Генерирует красивое имя файла по слот-системе."""
    
    # Слот 1: ALGO
    if algo == 'leiden_mod': a_tag = "LMod"
    elif algo.startswith('leiden_cpm'): 
        # leiden_cpm_0.05 -> LCPM05
        res = algo.split('_')[-1].replace('0.', '')
        a_tag = f"LCPM{res}"
    elif algo == 'infomap': a_tag = "Info"
    elif algo == 'label_prop': a_tag = "LProp"
    elif algo == 'walktrap': a_tag = "Walk"
    elif algo == 'eigenvector': a_tag = "Eigen"
    else: a_tag = algo

    # Слот 2: GRAPH
    # jaccard, 0.02 -> Jac02
    g_tag = f"{graph_type[:3].capitalize()}{str(edge_th).replace('0.', '')}"

    # Слот 3: SPARS
    # knn, 20 -> KNN20
    if spars_type == 'knn': s_tag = f"KNN{spars_param}"
    elif spars_type == 'backbone': s_tag = f"BB{str(spars_param).replace('0.', '')}"
    else: s_tag = "NoSpars"

    # Слот 4: TRACK
    t_tag = f"Tr{str(track_th).replace('0.', '')}"

    return f"{a_tag}_{g_tag}_{s_tag}_{t_tag}"

In [ ]:
# ================= МЕНЮ ЭКСПЕРИМЕНТОВ =================

# Сценарий 1: "Битва Титанов" (Сравнение основных подходов на стандартных настройках)
# Гипотеза: Infomap даст лучшее семантическое качество, Leiden Modularity - лучшую стабильность.
EXPERIMENTS = [
    # (Algo, GraphMethod, EdgeTh, SparsType, SparsParam)
    ("leiden_mod", "jaccard", 0.02, "knn", 20),
    ("infomap",    "jaccard", 0.02, "knn", 20),
    ("label_prop", "jaccard", 0.02, "knn", 20), # Baseline
]

# Сценарий 2: "Микроскоп" (Leiden CPM с разным разрешением)
# Гипотеза: CPM 0.05 найдет крупные жанры, CPM 0.1 найдет узкие фандомы.
# EXPERIMENTS = [
#     ("leiden_cpm_0.01", "jaccard", 0.02, "knn", 20), # Крупные куски
#     ("leiden_cpm_0.05", "jaccard", 0.02, "knn", 20), # Средние (Рекомендую)
#     ("leiden_cpm_0.10", "jaccard", 0.02, "knn", 20), # Мелкие детали
# ]

# Сценарий 3: "Топология" (Влияние прореживания)
# Гипотеза: Backbone лучше сохраняет структуру "звезд", KNN лучше связывает ниши.
# EXPERIMENTS = [
#     ("leiden_mod", "jaccard", 0.02, "knn", 10),      # Жесткий KNN
#     ("leiden_mod", "jaccard", 0.02, "knn", 30),      # Мягкий KNN
#     # ("leiden_mod", "jaccard", 0.02, "backbone", 0.05), # Backbone (если реализован)
# ]

# Сценарий 4: "Природа связей" (Jaccard vs Raw)
# Гипотеза: Raw покажет, что все смотрят Наруто. Jaccard покажет реальные вкусы.
# EXPERIMENTS = [
#     ("leiden_mod", "jaccard", 0.05, "knn", 20), # Высокий порог Жаккара
#     ("leiden_mod", "raw",     2,    "knn", 20), # Сырые пересечения > 2 юзеров
# ]

# Общий параметр трекинга для всех (можно вынести, так как он редко меняется)
TRACK_THRESH = 0.15

In [ ]:
# --- RUNNER ---

# Кэш для графов, чтобы не пересобирать их, если меняется только алгоритм кластеризации
# Структура: graph_cache[(method, edge_th, spars_type, spars_k)][year] = igraph_object
# Если памяти мало - убери кеширование и пересобирай каждый раз.
graph_cache = {} 

for algo, g_method, edge_th, s_type, s_param in EXPERIMENTS:
    
    # 1. Генерируем имя
    EXP_KEY = generate_exp_key(algo, g_method, edge_th, s_type, s_param, TRACK_THRESH)
    print(f"\n\n🚀 STARTING: {EXP_KEY}")
    
    history = {}
    
    for year in range(2006, 2012):
        
        # Ключ для кеша графа (граф зависит только от этих параметров, не от алгоритма кластеризации)
        cache_key = (g_method, edge_th, s_type, s_param)
        
        # Проверяем кеш
        if year in graph_cache.get(cache_key, {}):
            h = graph_cache[cache_key][year]
            # print(f"  [Year {year}] Loaded graph from cache.")
        else:
            # Если нет в кеше — строим
            print(f"  [Year {year}] Building graph...", end=" ")
            raw_edges, counts = builder.build_edges(year, max_users=20000, method=g_method)
            G = builder.build_graph(raw_edges, node_counts=counts, weight_threshold=edge_th)
            
            if s_type == "knn":
                G_clean = builder.sparsify_knn(G, k=s_param)
            elif s_type == "backbone":
                # G_clean = builder.sparsify_backbone_graph(G, alpha=s_param)
                pass
            else:
                G_clean = G
                
            if G_clean.number_of_nodes() == 0:
                print("Empty.")
                continue
                
            h = ig.Graph.TupleList(G_clean.edges(data=True), directed=False)
            
            # Сохраняем в кеш
            if cache_key not in graph_cache: graph_cache[cache_key] = {}
            graph_cache[cache_key][year] = h
            print("Done.")

        # Достаем веса
        weights = h.es['weight'] if 'weight' in h.edge_attributes() else None
        
        # КЛАСТЕРИЗАЦИЯ
        partition = get_partition(h, algo, weights=weights)
        
        if partition:
            history[year] = CommunityTracker.get_membership(h, partition)

    # --- POST PROCESSING ---
    print(f"  🏁 Finishing {EXP_KEY}...")
    
    # 1. Tracking
    aligned = CommunityTracker.track_communities(history, threshold=TRACK_THRESH)
    
    # 2. Saving
    csv_path = f"reports/partitions/{EXP_KEY}.csv"
    CommunityTracker.save_aligned_history_to_csv(aligned, csv_path)
    
    # 3. Metrics
    evaluator = ClusterEvaluation(EXP_KEY, aligned, anime_info_clean)
    stats = evaluator.evaluate()
    # Сохраняем метрики (можно append в общий csv)
    pd.DataFrame([stats]).to_csv("reports/all_metrics.csv", mode='a', header=not os.path.exists("reports/all_metrics.csv"), index=False)
    
    # 4. Sankey
    df_viz = enricher.enrich_partition(csv_path)
    ClusterVisualizer(df_viz).plot_evolution_sankey(
        f"reports/plots/Sankey_{EXP_KEY}.html", min_link_size=5, title=EXP_KEY
    )
    
    # Чистим память от ненужного
    del aligned, df_viz, evaluator, stats
    gc.collect()

print("\nAll experiments done.")

In [ ]:
# 1. TRACKING (Сшиваем года)
aligned_data = CommunityTracker.track_communities(partition_history, threshold=TRACKING_THRESH)

# 2. SAVING CSV
csv_filename = f"{REPORTS_DIR}/partitions/{EXP_KEY}.csv"
CommunityTracker.save_aligned_history_to_csv(aligned_data, csv_filename)

# 3. EVALUATION (Считаем метрики)
evaluator = ClusterEvaluation(EXP_KEY, aligned_data, anime_info_clean)
stats = evaluator.evaluate()

print("\n📊 SCOREBOARD:")
print(pd.Series(stats))

# Сохраняем метрики в общий файл (append mode)
metrics_file = f"{REPORTS_DIR}/all_experiments_metrics.csv"
df_stats = pd.DataFrame([stats])
if os.path.exists(metrics_file):
    df_stats.to_csv(metrics_file, mode='a', header=False, index=False)
else:
    df_stats.to_csv(metrics_file, index=False)

In [ ]:
# 1. Enrich (Добавляем названия и жанры)
df_viz = enricher.enrich_partition(csv_filename)

# 2. Visualize
viz = ClusterVisualizer(df_viz)
html_path = f"{REPORTS_DIR}/plots/Sankey_{EXP_KEY}.html"

# min_link_size=10 убирает тонкие линии (шум)
viz.plot_evolution_sankey(html_path, min_link_size=10, title=f"Evolution: {EXP_KEY}")

print(f"Open report: {html_path}")